In [1]:
import pandas as pd
from collections import defaultdict
import warnings
import visualize
import neat
import multiprocessing
import os
import matplotlib.pyplot as plt

import numpy as np

In [2]:
necoc = 100
generations = int(3000/necoc)
generations

30

In [3]:
def softmax(x, axis=1):
    # 计算每行的最大值
    row_max = x.max(axis=axis)
 
    # 每行元素都需要减去对应的最大值，否则求exp(x)会溢出，导致inf情况
    row_max=row_max.reshape(-1, 1)
    x = x - row_max
 
    # 计算e的指数次幂
    x_exp = np.exp(x)
    x_sum = np.sum(x_exp, axis=axis, keepdims=True)
    s = x_exp / x_sum
    return s

In [4]:
df = pd.read_csv('letter-recognition.data',header=None)

train_df = df[:1600]
test_df = df[16000:16000+400]

X = train_df.ix[:,1:16].to_numpy() 
y = np.array([ord(l)-ord('A') for l in train_df.ix[:,0].to_numpy()])

testX = test_df.ix[:,1:16].to_numpy() 
testy = np.array([ord(l)-ord('A') for l in test_df.ix[:,0].to_numpy()])

In [5]:
def get_winner(class1):
    
    numbers = [*range(26)] # number 2 > number 1
    number_of_classification = len(numbers)
    #number_of_sampling = 100

    list_y = y.tolist()
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

        
    class2 = []

    class1_indexes = []
    class2_indexes = []

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)

    print("number of class1:{}".format(len(class1_label)))
    print("number of class2:{}".format(len(class2_label)))

    samplesize = 1600
    x_inputs =[tuple(c) for c in X[class1_indexes].tolist()] + [tuple(c) for c in X[class2_indexes].tolist()]
    x_outputs = [tuple([c]) for c in class1_label + class2_label]
    
    
    def eval_genomes(genomes, config):
        for genome_id, genome in genomes:
            net = neat.nn.FeedForwardNetwork.create(genome, config)

            outputs = []
            for xi in x_inputs:
                output = net.activate(xi)
                outputs.append(output)


            px_outputs = softmax(np.array(outputs).reshape(samplesize, 2), axis=1)
            # the index of maximum in each line
            pred_outputs = np.argmax(px_outputs, axis = 1)
            real_outputs = np.array(x_outputs).reshape(samplesize,)

            acc = np.sum(pred_outputs == real_outputs)/samplesize

            genome.fitness = acc

    def run(config_file):
        # Load configuration.
        config = neat.Config(
            neat.DefaultGenome,
            neat.DefaultReproduction,
            neat.DefaultSpeciesSet,
            neat.DefaultStagnation,
            config_file,
        )

        # Create the population, which is the top-level object for a NEAT run.
        p = neat.Population(config)

        # add a stdout reporter to show progress in the terminal
        reporter = neat.StdOutReporter(False)
        p.add_reporter(reporter)
        stats = neat.StatisticsReporter()
        p.add_reporter(stats)
        #checkpointer = neat.Checkpointer(100)
        #p.add_reporter(checkpointer)
        # Run for up to 300 generations.
        winner = p.run(eval_genomes, generations)

        return [stats, winner]
    
    local_dir = os.getcwd()
    config_path = os.path.join(local_dir, "config-feedforward-letter2")
    [stats, winner] = run(config_path)
    
    return winner

In [6]:
import numpy as np
import random

number_codes = []
for i in range(2**(26-1), 2**26-1):
    if (i % 1000000) == 0:
            print(i)
    for j in bin(i).split('b')[1]:
        number_codes.append(int(j))
        


number_codes = np.array(number_codes).reshape(2**(26 - 1) - 1,26).T     

slice = random.sample(range(2**(26 - 1) - 1), necoc)

number_codes = number_codes[:,slice].tolist()

classes = []
for i in range(necoc):
    code_list = np.array(number_codes).T[i]
    classes.append(np.where(code_list==1)[0].tolist())

matrix_errors = []

for i in range(25):
    for j in range(i+1,26):
        matrix_errors.append(necoc - np.sum(np.array(number_codes[i]) == np.array(number_codes[j])))
min_error = np.min(matrix_errors)

print(min_error)

34000000
35000000
36000000
37000000
38000000
39000000
40000000
41000000
42000000
43000000
44000000
45000000
46000000
47000000
48000000
49000000
50000000
51000000
52000000
53000000
54000000
55000000
56000000
57000000
58000000
59000000
60000000
61000000
62000000
63000000
64000000
65000000
66000000
67000000
36


In [7]:
winner_list = []

for i,class1 in enumerate(classes):
    print("running:{}/30".format(i))
    winner = get_winner(class1)
    winner_list.append(winner)

running:0/30
number of class1:725
number of class2:875

 ****** Running generation 0 ****** 

Population's average fitness: 0.49705 stdev: 0.04424
Best fitness: 0.57875 - size: (2, 3) - species 5 - id 188
Average adjusted fitness: 0.114
Mean genetic distance 2.633, standard deviation 0.502
Population of 202 members in 12 species
Total extinctions: 0
Generation time: 2.738 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.52277 stdev: 0.03756
Best fitness: 0.57875 - size: (2, 3) - species 4 - id 188
Average adjusted fitness: 0.084
Mean genetic distance 2.590, standard deviation 0.511
Population of 201 members in 13 species
Total extinctions: 0
Generation time: 2.943 sec (2.841 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.52361 stdev: 0.04029
Best fitness: 0.61000 - size: (4, 7) - species 4 - id 550
Average adjusted fitness: 0.114
Mean genetic distance 2.607, standard deviation 0.517
Population of 203 members in 13 species
Total

Population's average fitness: 0.54037 stdev: 0.05003
Best fitness: 0.63313 - size: (4, 7) - species 14 - id 3287
Average adjusted fitness: 0.095
Mean genetic distance 3.086, standard deviation 0.552
Population of 207 members in 22 species
Total extinctions: 0
Generation time: 5.385 sec (5.080 average)

 ****** Running generation 25 ****** 

Population's average fitness: 0.53623 stdev: 0.05185
Best fitness: 0.63313 - size: (4, 7) - species 14 - id 3287
Average adjusted fitness: 0.093
Mean genetic distance 3.077, standard deviation 0.540
Population of 204 members in 22 species
Total extinctions: 0
Generation time: 5.410 sec (5.047 average)

 ****** Running generation 26 ****** 

Population's average fitness: 0.53840 stdev: 0.05231
Best fitness: 0.63313 - size: (4, 7) - species 35 - id 3287
Average adjusted fitness: 0.089
Mean genetic distance 3.102, standard deviation 0.544
Population of 193 members in 20 species
Total extinctions: 0
Generation time: 5.401 sec (5.075 average)

 ****** Ru

Population's average fitness: 0.53729 stdev: 0.04185
Best fitness: 0.63562 - size: (4, 10) - species 27 - id 1322
Average adjusted fitness: 0.108
Mean genetic distance 2.994, standard deviation 0.494
Population of 204 members in 38 species
Total extinctions: 0
Generation time: 4.762 sec (4.468 average)

 ****** Running generation 19 ****** 

Population's average fitness: 0.53712 stdev: 0.04130
Best fitness: 0.63562 - size: (4, 10) - species 27 - id 1322
Average adjusted fitness: 0.109
Mean genetic distance 2.975, standard deviation 0.502
Population of 191 members in 39 species
Total extinctions: 0
Generation time: 4.616 sec (4.528 average)

 ****** Running generation 20 ****** 

Population's average fitness: 0.54394 stdev: 0.04374
Best fitness: 0.65063 - size: (9, 18) - species 40 - id 2987
Average adjusted fitness: 0.114
Mean genetic distance 2.968, standard deviation 0.498
Population of 188 members in 39 species
Total extinctions: 0
Generation time: 4.546 sec (4.568 average)

 ******

Population's average fitness: 0.55788 stdev: 0.09113
Best fitness: 0.66875 - size: (3, 4) - species 21 - id 1203
Average adjusted fitness: 0.158
Mean genetic distance 2.974, standard deviation 0.607
Population of 201 members in 23 species
Total extinctions: 0
Generation time: 4.189 sec (3.852 average)

 ****** Running generation 13 ****** 

Population's average fitness: 0.55081 stdev: 0.09159
Best fitness: 0.66875 - size: (3, 4) - species 21 - id 1203
Average adjusted fitness: 0.143
Mean genetic distance 3.052, standard deviation 0.580
Population of 203 members in 23 species
Total extinctions: 0
Generation time: 4.152 sec (3.940 average)

 ****** Running generation 14 ****** 

Population's average fitness: 0.56275 stdev: 0.08777
Best fitness: 0.67688 - size: (3, 4) - species 21 - id 2491
Average adjusted fitness: 0.155
Mean genetic distance 3.121, standard deviation 0.570
Population of 204 members in 24 species
Total extinctions: 0
Generation time: 4.308 sec (3.980 average)

 ****** Ru

Population's average fitness: 0.52938 stdev: 0.05001
Best fitness: 0.62000 - size: (4, 5) - species 5 - id 494
Average adjusted fitness: 0.120
Mean genetic distance 2.920, standard deviation 0.511
Population of 199 members in 21 species
Total extinctions: 0
Generation time: 3.506 sec (3.168 average)

 ****** Running generation 7 ****** 

Population's average fitness: 0.53025 stdev: 0.04867
Best fitness: 0.62000 - size: (4, 5) - species 5 - id 494
Average adjusted fitness: 0.108
Mean genetic distance 2.923, standard deviation 0.525
Population of 200 members in 24 species
Total extinctions: 0
Generation time: 3.586 sec (3.220 average)

 ****** Running generation 8 ****** 

Population's average fitness: 0.53258 stdev: 0.05121
Best fitness: 0.62250 - size: (5, 6) - species 8 - id 1515
Average adjusted fitness: 0.097
Mean genetic distance 2.962, standard deviation 0.503
Population of 204 members in 26 species
Total extinctions: 0
Generation time: 3.801 sec (3.284 average)

 ****** Running g

...

Population's average fitness: 0.56050 stdev: 0.09340
Best fitness: 0.66812 - size: (3, 5) - species 6 - id 690
Average adjusted fitness: 0.181
Mean genetic distance 3.013, standard deviation 0.541
Population of 206 members in 30 species
Total extinctions: 0
Generation time: 6.375 sec (7.189 average)

 ****** Running generation 13 ****** 

Population's average fitness: 0.55187 stdev: 0.09828
Best fitness: 0.66812 - size: (3, 5) - species 6 - id 690
Average adjusted fitness: 0.176
Mean genetic distance 3.027, standard deviation 0.546
Population of 205 members in 31 species
Total extinctions: 0
Generation time: 6.218 sec (7.259 average)

 ****** Running generation 14 ****** 

Population's average fitness: 0.54596 stdev: 0.10026
Best fitness: 0.66812 - size: (3, 5) - species 6 - id 690
Average adjusted fitness: 0.160
Mean genetic distance 3.009, standard deviation 0.558
Population of 204 members in 31 species
Total extinctions: 0
Generation time: 6.270 sec (7.129 average)

 ****** Running 

Population's average fitness: 0.53237 stdev: 0.04352
Best fitness: 0.60437 - size: (2, 3) - species 14 - id 37
Average adjusted fitness: 0.120
Mean genetic distance 3.021, standard deviation 0.514
Population of 202 members in 30 species
Total extinctions: 0
Generation time: 5.577 sec (4.527 average)

 ****** Running generation 7 ****** 

Population's average fitness: 0.53464 stdev: 0.04527
Best fitness: 0.60437 - size: (2, 3) - species 14 - id 37
Average adjusted fitness: 0.123
Mean genetic distance 3.036, standard deviation 0.537
Population of 194 members in 31 species
Total extinctions: 0
Generation time: 5.110 sec (4.600 average)

 ****** Running generation 8 ****** 

Population's average fitness: 0.53503 stdev: 0.04572
Best fitness: 0.60437 - size: (2, 3) - species 14 - id 37
Average adjusted fitness: 0.109
Mean genetic distance 3.014, standard deviation 0.541
Population of 199 members in 31 species
Total extinctions: 0
Generation time: 5.091 sec (4.655 average)

 ****** Running ge

number of class2:771

 ****** Running generation 0 ****** 

Population's average fitness: 0.49992 stdev: 0.03433
Best fitness: 0.58625 - size: (2, 3) - species 1 - id 9
Average adjusted fitness: 0.162
Mean genetic distance 2.744, standard deviation 0.544
Population of 199 members in 18 species
Total extinctions: 0
Generation time: 3.403 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.51132 stdev: 0.03480
Best fitness: 0.60313 - size: (3, 5) - species 3 - id 277
Average adjusted fitness: 0.157
Mean genetic distance 2.937, standard deviation 0.575
Population of 199 members in 20 species
Total extinctions: 0
Generation time: 4.009 sec (3.706 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.51393 stdev: 0.03479
Best fitness: 0.63687 - size: (3, 5) - species 3 - id 532
Average adjusted fitness: 0.104
Mean genetic distance 2.948, standard deviation 0.585
Population of 199 members in 22 species
Total extinctions: 0
Generation time: 4.0

Population's average fitness: 0.55171 stdev: 0.05885
Best fitness: 0.68625 - size: (8, 16) - species 13 - id 3527
Average adjusted fitness: 0.183
Mean genetic distance 3.143, standard deviation 0.564
Population of 201 members in 31 species
Total extinctions: 0
Generation time: 6.030 sec (6.362 average)

 ****** Running generation 25 ****** 

Population's average fitness: 0.55689 stdev: 0.05638
Best fitness: 0.68625 - size: (8, 16) - species 13 - id 3527
Average adjusted fitness: 0.090
Mean genetic distance 3.141, standard deviation 0.554
Population of 192 members in 29 species
Total extinctions: 0
Generation time: 6.418 sec (6.341 average)

 ****** Running generation 26 ****** 

Population's average fitness: 0.55407 stdev: 0.05899
Best fitness: 0.68625 - size: (8, 16) - species 13 - id 3527
Average adjusted fitness: 0.169
Mean genetic distance 3.152, standard deviation 0.565
Population of 191 members in 28 species
Total extinctions: 0
Generation time: 6.156 sec (6.336 average)

 ******

Population's average fitness: 0.55815 stdev: 0.07094
Best fitness: 0.65250 - size: (5, 7) - species 8 - id 2290
Average adjusted fitness: 0.158
Mean genetic distance 3.057, standard deviation 0.590
Population of 202 members in 21 species
Total extinctions: 0
Generation time: 4.676 sec (4.780 average)

 ****** Running generation 19 ****** 

Population's average fitness: 0.55738 stdev: 0.07408
Best fitness: 0.66625 - size: (3, 6) - species 20 - id 3000
Average adjusted fitness: 0.177
Mean genetic distance 3.098, standard deviation 0.594
Population of 196 members in 19 species
Total extinctions: 0
Generation time: 5.066 sec (4.839 average)

 ****** Running generation 20 ****** 

Population's average fitness: 0.55690 stdev: 0.07474
Best fitness: 0.66625 - size: (3, 6) - species 20 - id 3000
Average adjusted fitness: 0.155
Mean genetic distance 3.086, standard deviation 0.599
Population of 193 members in 18 species
Total extinctions: 0
Generation time: 4.905 sec (4.865 average)

 ****** Run

In [8]:
local_dir = os.getcwd()
config_path = os.path.join(local_dir, "config-feedforward-letter2")

config = neat.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    config_path,
)

def get_pred_real(i, j):
    winner_net = neat.nn.FeedForwardNetwork.create(winner_list[i], config)
    
    numbers = [*range(26)]
    class1 = classes[i]
    class2 = []
    class1_indexes = []
    class2_indexes = []
    
    test_X = testX[j]
    test_y = testy[j]

    list_y = [test_y]
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)


    testsamplesize = 1
    test_x_inputs =[tuple(test_X)]
    test_x_outputs = [tuple([c]) for c in class1_label + class2_label]

    outputs = []
    for xi in test_x_inputs:
        output = winner_net.activate(xi)
        outputs.append(output)

    px_outputs = softmax(np.array(outputs).reshape(testsamplesize, 2), axis=1)
    # the index of maximum in each line
    pred_outputs = np.argmax(px_outputs, axis = 1)
    real_outputs = np.array(test_x_outputs).reshape(testsamplesize,)

    
    return [pred_outputs, real_outputs]

from random import randint
pred_value = []
error_list = []

for j in range(400):
    pred = []
    for i in range(necoc):
        [pred_outputs, real_outputs] = get_pred_real(i, j)
        #print(pred_outputs, real_outputs)
        pred.append(pred_outputs)
    #print(np.array(pred).T)
    
    error = []
    for i in range(26):
        error.append(necoc - np.sum(number_codes[i] == np.array(pred).T))
    #print(error)
    
    pred_value.append(np.where(error==np.min(error)))
    error_list.append(np.min(error))
    #print(np.where(error==np.min(error)) )
list_P = []
for i in pred_value:
    if (len(i[0])) == 1:
        list_P.append(i[0][0])
    else:
        random_pick = randint(0, len(i[0])-1)
        list_P.append(i[0][random_pick])
np.sum(list_P == testy)/400

0.45

In [9]:
def get_pred_train(i, j):
    winner_net = neat.nn.FeedForwardNetwork.create(winner_list[i], config)
    
    numbers = [*range(26)]
    class1 = classes[i]
    class2 = []
    class1_indexes = []
    class2_indexes = []
    
    test_X = X[j]
    test_y = y[j]

    list_y = [test_y]
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)


    testsamplesize = 1
    test_x_inputs =[tuple(test_X)]
    test_x_outputs = [tuple([c]) for c in class1_label + class2_label]

    outputs = []
    for xi in test_x_inputs:
        output = winner_net.activate(xi)
        outputs.append(output)

    px_outputs = softmax(np.array(outputs).reshape(testsamplesize, 2), axis=1)
    # the index of maximum in each line
    pred_outputs = np.argmax(px_outputs, axis = 1)
    real_outputs = np.array(test_x_outputs).reshape(testsamplesize,)

    
    return [pred_outputs, real_outputs]

pred_value = []
error_list = []

for j in range(1600):
    pred = []
    for i in range(necoc):
        [pred_outputs, real_outputs] = get_pred_train(i, j)
        #print(pred_outputs, real_outputs)
        pred.append(pred_outputs)
    #print(np.array(pred).T)
    
    error = []
    for i in range(26):
        error.append(26 - np.sum(number_codes[i] == np.array(pred).T))
    #print(error)
    
    pred_value.append(np.where(error==np.min(error)))
    error_list.append(np.min(error))
    #print(np.where(error==np.min(error)) )

list_P = []
for i in pred_value:
    if (len(i[0])) == 1:
        list_P.append(i[0][0])
    else:
        random_pick = randint(0, len(i[0])-1)
        list_P.append(i[0][random_pick])
print("Trainset acc:{}".format(np.sum(list_P == y)/1600))
winner_fitness = []
for winner in winner_list:
    #print(winner.fitness)
    winner_fitness.append(winner.fitness)
print("Avg Base Learner:{}".format(np.mean(winner_fitness)))

Trainset acc:0.5
Avg Base Learner:0.6730937499999999


In [10]:
# program to check if there is exist a path between two vertices 
# of a graph 

from collections import defaultdict 

#This class represents a directed graph using adjacency list representation 
class Graph: 

	def __init__(self,vertices): 
		self.V= vertices #No. of vertices 
		self.graph = defaultdict(list) # default dictionary to store graph 

	# function to add an edge to graph 
	def addEdge(self,u,v): 
		self.graph[u].append(v) 
	
	# Use BFS to check path between s and d 
	def isReachable(self, s, d): 
		# Mark all the vertices as not visited 
		visited =[False]*(self.V) 

		# Create a queue for BFS 
		queue=[] 

		# Mark the source node as visited and enqueue it 
		queue.append(s) 
		visited[s] = True

		while queue: 

			#Dequeue a vertex from queue 
			n = queue.pop(0) 
			
			# If this adjacent node is the destination node, 
			# then return true 
			if n == d: 
				return True

			# Else, continue to do BFS 
			for i in self.graph[n]: 
				if visited[i] == False: 
					queue.append(i) 
					visited[i] = True
		# If BFS is complete without visited d 
		return False
    
    
def findAllPath(graph,start,end,path=[]):
    path = path +[start]
    if start == end:
        return [path]
 
    paths = [] #存储所有路径    
    for node in graph[start]:
        if node not in path:
            newpaths = findAllPath(graph,node,end,path) 
            for newpath in newpaths:
                paths.append(newpath)
    return paths

numbers = [*range(2)]
number_of_classification = len(numbers)

list_nodes_number = []
list_connection_number = []

for winner in winner_list:
    used_nodes = list(winner.nodes.keys())
    # create a dict for mapping 
    l1 = numbers + config.genome_config.input_keys + used_nodes[number_of_classification:]
    l2 = range(len(l1))
    dict_nodes = dict(zip(l1, l2))

    ### 将节点加入图
    g = Graph(len(l1))# inputs + outputs + used  
    # add connections in the graph
    for cg in winner.connections.values():
        if cg.enabled:
            g.addEdge(dict_nodes[cg.key[0]],dict_nodes[cg.key[1]])

    v = numbers
    list_connections = []
    for vi in v:
        for u in range(number_of_classification, len(config.genome_config.input_keys) + number_of_classification):
            list_connections.append(g.isReachable(u, vi))

    nodes_inputs = range(number_of_classification, number_of_classification+ 64)
    nodes_outputs = range(number_of_classification)
    nodes_mid = range(number_of_classification + 64, len(dict_nodes))

    # 找到所有输入和输出连接的路
    all_path = []
    for u in nodes_inputs:
        for v in nodes_outputs:
            path = findAllPath(g.graph, u, v)
            if path:
                all_path = all_path + path

    # 得到最长路，以及每个节点在哪个层级
    max_length = max([len(x) for x in all_path])
    nodes_tuples_list = []
    for path in all_path:
        #print(path)
        for node in path:
            nodes_tuples_list.append([node, path.index(node)])

    # 确定节点的层级（消除重复）       
    nodes_tuples_fixed_list = []
    for index in range(max_length):
        for nodes in nodes_tuples_list:
            node, node_index = nodes[0], nodes[1]
            if node_index == index: 
                if nodes not in nodes_tuples_fixed_list:
                    nodes_tuples_fixed_list.append(nodes)


    # 所有有连接的节点
    all_used_nodes = []
    for node in nodes_tuples_fixed_list:
        if node[0] not in all_used_nodes:
            all_used_nodes.append(node[0])

    # 确定节点的层级（最终确定）
    nodes_tuples_fixed_list_final = []

    for node in all_used_nodes:
        if node in nodes_inputs:
            nodes_tuples_fixed_list_final.append([node, 0])
        elif node in nodes_outputs:
            nodes_tuples_fixed_list_final.append([node, max_length - 1])
        else:
            list_node = []
            for nodes in nodes_tuples_fixed_list:
                if node == nodes[0]:
                    list_node.append(nodes)
            layer = 0
            for ele in list_node:
                # 删除重复项，取最大
                if ele[1] > layer:
                    layer = ele[1]
            nodes_tuples_fixed_list_final.append([node,layer])


    # 每个层级的节点合数
    length_of_layers = []
    number_of_nodes = 0
    for index in range(max_length):
        for nodes in nodes_tuples_fixed_list_final:
            node, node_index = nodes[0], nodes[1]
            if node_index == index: 
                number_of_nodes += 1
        length_of_layers.append(number_of_nodes)

    length_of_each_layer = []
    length_of_each_layer.append(length_of_layers[0])
    for i in range(1, max_length):
        length_of_each_layer.append(length_of_layers[i] - length_of_layers[i-1])

    # 输出每个层级的节点个数
    #print("length of each layers:", length_of_each_layer)

    # 所有端到端的路
    all_path_side2side = []
    for path in all_path:
        if len(path) == 2:
            all_path_side2side.append(path)
        else:
            for i in range(len(path)-1):
                all_path_side2side.append([path[i],path[i+1]])

    # 定义节点到绘图的字典
    dict_nodes_graph = dict()
    count_number_layer = [0] * max_length
    for nodes in nodes_tuples_fixed_list_final:
        node, layer = nodes
        dict_nodes_graph[node] = [count_number_layer[layer], layer]
        count_number_layer[layer] += 1
    list_nodes_number.append(np.sum(length_of_each_layer))
    list_connection_number.append(len(all_path_side2side))
    #print("Number of nodes:{} Number of connections:{}".format(np.sum(length_of_each_layer),len(all_path_side2side)))
print("Total nodes:{} Total connections::{}".format(np.sum(list_nodes_number), np.sum(list_connection_number)))

Total nodes:1083 Total connections::2384


In [11]:
local_dir = os.getcwd()
config_path = os.path.join(local_dir, "config-feedforward-letter2")

config = neat.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    config_path,
)

def get_pred_real(i, j):
    winner_net = neat.nn.FeedForwardNetwork.create(winner_list[i], config)
    
    numbers = [*range(26)]
    class1 = classes[i]
    class2 = []
    class1_indexes = []
    class2_indexes = []
    
    test_X = testX[j]
    test_y = testy[j]

    list_y = [test_y]
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)


    testsamplesize = 1
    test_x_inputs =[tuple(test_X)]
    test_x_outputs = [tuple([c]) for c in class1_label + class2_label]

    outputs = []
    for xi in test_x_inputs:
        output = winner_net.activate(xi)
        outputs.append(output)

    px_outputs = softmax(np.array(outputs).reshape(testsamplesize, 2), axis=1)
    
    # the index of maximum in each line
    return px_outputs

In [12]:
from random import randint
pred_value = []
error_list = []

for j in range(400):
    pred = []
    for i in range(necoc):
        pred_outputs = get_pred_real(i, j)
        pred.append(pred_outputs)

    error = []
    for i in range(26):
        loss = 0
        for ii in range(len(pred)):
            #print(number_codes[0][ii], pred[ii][0])
            #print (pred[ii][0][number_codes[0][ii]])
            loss += (1 - pred[ii][0][number_codes[i][ii]])**2
        error.append(loss)

    pred_value.append(np.where(error==np.min(error)))

list_P = []
for i in pred_value:
    if (len(i[0])) == 1:
        list_P.append(i[0][0])
    else:
        random_pick = randint(0, len(i[0])-1)
        list_P.append(i[0][random_pick])
np.sum(list_P == testy)/400

0.36